# Second Assignment for the CE889 - Neural Networks course

## Group #18

### Team members:

Sudeep Sawant

Francisco Sánchez

James Scoon

### The first step of the assignment was to import the necessary libraries:

### Here, the training data is split into input and output variables using numpy:

In [ ]:
X = training_store.drop(columns=["Sales", "Customers"]).to_numpy()
y = training_store['Sales'].to_numpy()
X_test = testing_store.drop(columns=["Id"]).to_numpy()

#### The input values are then normalised

We tried normalising the output, but we noted that it had a worse effect on the learning of the network than leaving the data alone. We believe this is because there are a lot of columns with zeroes, and by normalising it is easier for the network to predict only zeroes.

In [22]:
inputScaler = MinMaxScaler()
inputScaler.fit(X)
X = inputScaler.transform(X)
X_test = inputScaler.transform(X_test)
#y = y.reshape(-1, 1)
#outputScaler = MinMaxScaler()
#outputScaler.fit(y)
#y = outputScaler.transform(y)

### To make the data fit the CNN, we reshape it into multiple dimensions

The training data is separated into 942 tables, each one corresponding to one consecutive day of the year. These tables have 1115 rows (one for each store) with 27 columns of features.

The idea is that the CNN uses a filter to compare similar stores over different days to predict the sales.

The test data is only split into 48 days of tables.

In [23]:
X = np.reshape(X, (942, 1115, 27))
y = np.reshape(y, (942, 1115))
X_test = np.reshape(X_test, (48, 1115, 27))
X = np.expand_dims(X, axis = 3)
X_test = np.expand_dims(X_test, axis = 3)

Here, we can see the shape of the data is consistent with what we want

In [24]:
print("X:\t", X.shape)
print("y:\t", y.shape)
print("X_test:\t", X_test.shape)

X:	 (942, 1115, 27, 1)
y:	 (942, 1115)
X_test:	 (48, 1115, 27, 1)


### Finally, the data can be split into training and validation sets

The data is split into 75% training data and 25% validation data

In [25]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25, random_state = 42, shuffle=True)

### The Neural Network is constructed in the following lines

We are using a deep convolutional neural network with a convolutional layer that contains 30 10x10 filters, which feeds into another convolutional layer with 25 4x4 filters. This layer is then sent into a 2x2 pooling layer. The result is then flattened and output into a set of 1115 output neurons (one for each store).

In [26]:
model = Sequential()

model.add(Conv2D(30, 10, input_shape = (1115, 27, 1)))
model.add(Conv2D(25, 4, input_shape = (1115, 27, 1)))
model.add(MaxPooling2D(pool_size = 2))
model.add(Flatten())
model.add(Dense(1115, activation='relu'))

#### Here, we use the Adam optimizer with a learning rate of 0.005 and a decay of 10^-6

We also use mean square error as the loss function

In [27]:
opt = keras.optimizers.Adam(lr=0.005, decay=1e-6)

model.compile(
    loss='mse',
    optimizer=opt,
    metrics=['accuracy'],
)

### We can finally train the network

The network is trained for 20 epochs with a batch size of 10. An early stopping criterion is used to stop the network if no progress is made. This criterion also returns the best set of weights that it found while training the network.

In [28]:
model.fit(X_train, y_train, epochs = 20, batch_size = 10, validation_data = (X_validation, y_validation), callbacks = [
    EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 3, verbose = 1, mode = 'auto', restore_best_weights = True)
])

Train on 706 samples, validate on 236 samples
Epoch 1/20
706/706 [==============================] - 133s 189ms/step - loss: 22394384.1841 - accuracy: 0.1742 - val_loss: 18719110.1525 - val_accuracy: 0.4661
Epoch 2/20
706/706 [==============================] - 132s 187ms/step - loss: 15064798.8244 - accuracy: 0.4490 - val_loss: 13134284.4661 - val_accuracy: 0.2076
Epoch 3/20
706/706 [==============================] - 128s 181ms/step - loss: 12121371.6813 - accuracy: 0.5241 - val_loss: 11768578.8305 - val_accuracy: 0.5932
Epoch 4/20
706/706 [==============================] - 127s 179ms/step - loss: 10826760.7295 - accuracy: 0.6487 - val_loss: 10782480.4322 - val_accuracy: 0.6949
Epoch 5/20
706/706 [==============================] - 126s 179ms/step - loss: 10299392.4235 - accuracy: 0.7153 - val_loss: 10463174.9788 - val_accuracy: 0.7034
Epoch 6/20
706/706 [==============================] - 124s 175ms/step - loss: 10070239.8839 - accuracy: 0.7054 - val_loss: 10230729.7415 - val_accuracy: 0

The validation accuracy is somewhat acceptable, at around 76%

The next step is to predict and format the output, by turning it into a single column of data (the sales)

In [29]:
y_test = model.predict(X_test)
#y_test = outputScaler.inverse_transform(model.predict(X_test))
output = np.reshape(y_test, 48 * 1115)

As the last step of the assignment, the results are saved to the submission.csv file. Because the test data had extra rows inserted with an Id of zero, we ignore these rows while exporting the results.

In [31]:
outputRows = []
for index in range(len(output)):
    if testing_store["Id"].iloc[index] == 0:
        continue
    outputRows.append(output[index])
outputFile = open("submission.csv", "w")
outputFile.write('"Id","Sales"\n')
for row in range(len(outputRows)):
    outputFile.write(f"{row + 1},{outputRows[row]}\n")
outputFile.close()